
### RTG 2175 Retreat Sylvenstein

# Testing and code quality

                                    Michael Sonntag

                                    Department Biologie II
                                    Ludwig-Maximilians-Universität München

                                    20.10.2021


![G-Node-logo.png](./resources/G-Node-logo.png)

## Testing code

- crashcourse in how to write tests in Python
- pointers how to write tests with Matlab

### Why test code

Controls! When conducting experiments, running controls is essential. Tests act like controls for your code.
  
- make sure code works as expected

- ensure code changes do not change the expected results

- ensure programming language or library package updates does not lead to different results

- document how code is used: point people to tests

### What's your gain from this session
- learn how to write and run simple tests
- get an idea how and where to start writing tests for your own projects


### Test frameworks
Could write your own test code without any additional packages. Its more convenient to use something thats already there.
There are already testframeworks out there that make testing more convenient and provide additional functionality
- `pytest`: https://docs.pytest.org
- `nose`: 
- `tox`: 

We will be working with pytest.

### Pytest requirements

```
pip install pytest
```

Find documentation here: https://docs.pytest.org

### Write and run basic tests

On the command line you can use pytest:

pytest 

Which tests are done?

pytest will 
- check all Python files in the current folder and all files in any subfolder.

- any file in a subfolder starting with "test_" will be run
- any file that is specifically handed to pytest will be run

- any function within a run file that starts with "test_" will be run

mkdir -p testlint/test
touch testlint/test/example.py
touch testlint/test/example_ignored.py
touch testlint/test/test_example.py
cd testlint

Function and test can be in the same file e.g. in example.py:

- file content of example.py
def add_up(a, b):
    return a + b

def test_add_up_succeed():
    assert add_up(1, 2) == 3

def test_add_up_fail():
    assert add_up(1, 2) == 4

def ignore_me():
    assert "I will be" == "completely ignored"

Run the test by providing the file name

pytest test/example.py

By default Python files are ignored even if they contain test functions:

- file content of example_ignored.py:
def ignored():
    assert "a" == "b"

def test_ignored():
    assert "b" == "c"

pytest tests

By default Python files with the "test_" prefix are run:

- file content of test_example.py:
def i_am_ignored():
    assert "a" == "b"


def test_i_am_included_success():
    assert 1 == 1


def test_i_am_included_fail():
    assert 1 == 2

pytest tests

Including files can be forced by providing the folder content:

pytest test/*


Great. We know now how to write test functions and how to run them.


How do we write tests that include functions from a package?

mkdir -p testlint
touch testlint/util.py
touch testlint/__init__.py
touch test/__init_.py
touch test/test_testlint_util.py

```
testlint
- testlint
  - __init__.py
  - util.py
- test
  - __init__.py
  - test_testlint_util.py
  - [previous test files]
```

testlint/util.py content:

def add_yourself(a):
    return a + a


testlint/__init__.py content

from . import util

test/test_testlint_util.py content:

import testlint


def test_add_yourself_success():
    assert testlint.util.add_yourself(1) == 2


def test_add_yourself_fail():
    assert testlint.util.add_yourself(1) == 3

and now we run the tests:

pytest test


Great, now we also know how to include functions from a local package.

If the package is pip installable, you can simply use plain imports after the package has been installed.

Using "assert" is nice. But it would even be nicer to have more options to test.

#### Using `unittest`

pytest supports the `unittest` package that gives many many additional options to write sophisticated tests. Why can this be nice:

Some of the nice features:
- gain more convenient "asserts", test exceptions raised
- while writing code, run only specific tests
- set up specific conditions before every test
- clean up after every test



Crashcourse:

touch test/test_unittest_example.py

test_unittest_example.py content:

import unittest

import testlint


class testutiladdyourself(unittest.TestCase):
    def setUp(self):
        self.setupvar = 2

    def test_add_yourself_success(self):
        self.assertEqual(testlint.util.add_yourself(self.setupvar), 4)

    def test_add_yourself_fail(self):
        self.assertEqual(testlint.util.add_yourself(self.setupvar), 3)

Run all tests from this class:

pytest test/test_unittest.py::testutiladdyourself

Run only a single test from this class:

pytest test/test_unittest.py::testutiladdyourself::test_add_yourself_success

For more details on what the unittest can do, again check the unittest documentation: https://docs.python.org/3/library/unittest.html
For a list of available asserts, check here: https://docs.python.org/3/library/unittest.html#test-cases:

Method|Checks that|New in
assertEqual(a, b)|a == b|
assertNotEqual(a, b)|a != b|
assertTrue(x)|bool(x) is True|
assertFalse(x)|bool(x) is False|
assertIs(a, b)|a is b|3.1
assertIsNot(a, b)|a is not b|3.1
assertIsNone(x)|x is None|3.1
assertIsNotNone(x)|x is not None|3.1
assertIn(a, b)|a in b|3.1
assertNotIn(a, b)|a not in b|3.1
assertIsInstance(a, b)|isinstance(a, b)|3.2
assertNotIsInstance(a, b)|not isinstance(a, b)|3.2

How nice. This gives us quite a lot of control over the tests we can write.

Tests can be written in varying granularity:
- write a test for each and every function you have in a code project -> you will stumble accross edge cases where the software does not work how you expect it or where it is outright bugged.
- but you are scientists; your main job is to get data and analyse it
- write a single test that shows that your code spits out the results you expect
  - e.g. select a raw data file where you KNOW what the results of your analysis should be. Keep this file around with your code. Write a test, that runs your code with THIS file and check that the results you get at the end of this test are what is expected.
  - run this test EVERYTIME you make changes to your code
- usually an analysis has multiple distinct steps with distinct result files.
  - write a test for every one of these distinct steps. this helps to narrow down if things break and to document how the individual steps should be run

- same when starting a new project ... write a test when the first useful result comes in and run the test everytime changes are made to the code or a new substep has been added.

- when developing you usually try it out until it works interactively; in ipython, in your IDE of choice, etc. Keep this code around! Copy your ipython history to a script file, add an assert at the very end and you have your first test -> this result should never change unexpectedly.


### Writing tests with Matlab

Matlab provides its own Test environment; you can find documentation on all different available tests here: https://de.mathworks.com/help/matlab/matlab-unit-test-framework.html

With newer Matlab version, it already comes with its own unit test environment. Simply write a file that contains test code and run it using the "runtest" command:

e.g.

Write a simple test script "testFile.m" with the content:

% assert success:
assert (1 == 1)

% assert fail:
assert(1 == 2)

and run the file:

result = runtests('testFile');

As far as I was able to identify, Matlab intruduced the full unit test framework to Matlab with the R2013a release. For any previous releases you can write your own minimal testing framework:

All tested code files and test files must be available to the Matlab path; the tests require the following files:
- tests/RunTests.m
- tests/wrapper.m
- tests/TestUtils.m
- tests/TestExample.m

The files require the following content:


tests/RunTests.m: 

```Matlab
%-- Runner for all other tests.
clear all;

stats.okCount = 0;
stats.errorCount = 0;

disp([10 'starting tests']);

%-- ToDo: create proper testfile, loose the 24mb one

%-- All current tests use the first block from file 'test.h5' with
%--     id: 7b59c0b9-b200-4b53-951d-6851dbd1cdc8
%--     name: joe097

all_tests = {};
all_tests{end+1} = struct('name', 'EXAMPLE', 'tests', {TestExample()});

for i = 1:length(all_tests)
    fprintf([10 'Execute ' all_tests{i}.name ' tests:\n\n']);
    
    for j = 1:length(all_tests{i}.tests)
        stats = wrapper(all_tests{i}.tests{j}, stats);
    end
end;

disp([10 'Tests: ' num2str(stats.okCount) ' succeeded, ' num2str(stats.errorCount) ' failed']);
```

tests/wrapper.m:
```
function stats = wrapper( func, stats )
%WRAPPER Wrapper function for every test that catches the exception
%   and prints out detaied report.

    try
        clearvars -except stats func; %-- ensure clean workspace

        func(); % execute unit test

        fprintf('Test %s ... OK\n', func2str(func));
        stats.okCount = stats.okCount + 1;
        clearvars -except stats func; %-- close handles
        
    catch me
        fprintf('Test %s ... ERROR\n', func2str(func));
        TestUtils.printErrorStack(me);
        stats.errorCount = stats.errorCount + 1;
    end;

end
```

tests/TestUtils.m
```

classdef TestUtils

    methods(Static)

        function printErrorStack(me)
            disp([9 me.message]);
            printStack = {me.stack(:).name; me.stack(:).file; me.stack(:).line}';
            disp(vertcat({'Name', 'File', 'Line'}, printStack));
        end;

    end;

end
```

tests/TestExample.m
```
% this file contains the actual tests

function funcs = TestExample
%TESTEXAMPLE runs example tests
%   Detailed explanation goes here

    funcs = {};
    funcs{end+1} = @test_add;
    funcs{end+1} = @test_subtract;
end

%% Test: Addition test
function [] = test_add( varargin )
    assert(1+1 == 2);
    assert(1+1 == 3);
end

%% Test: Subtraction test
function [] = test_subtract( varargin )
    assert(2-1 == 1);
end
```

The tests can be run by executing the "RunTests.m" script. There can be more than one Test file and every file can of course contain more than one test, but when adding files or tests the entries in "RunTests.m" or in the corresponding test file have to be updated.


## Increasing code quality

- why should we care

Keeping code organized and well maintained has multiple benefits. Most of these benefits lie in the future...
- have an easier time reading your own code:
  - keep your naming scheme consistent and include useful variable names so you know what the content is
  - using small functions or methods with fewer variables helps to grasp what they actually do
- have an easier time updating and rewriting your code
- have an easier time writing useful tests
  - having small functions help writing easy tests
- make it easier for other people to understand your code
- don't be hesitant to show other people your code because of quality
- write better code over time


General guidelines to write clean Python code - PEP8:
https://www.python.org/dev/peps/pep-0008/

These are many things to keep in mind and do not even touch "code smells". To help increase code quality and making it part of a routine, there are "linters" out there to make it easy for you. Examples are:
- pylint
- flake8
- autopep8
- pychecker
- pylama
- ...

We are going to use pylint (https://pylint.org/) which has a good balance between clean code styles and identifying code smells.

pip install pylint

run it from the command line

pylint pylint testlint/util.py

And now we work our way through all the issues...

You can deactivate specific messages


Create a file "setup.cfg" at the root of the project and paste the following:
```
[pycodestyle]
max-line-length = 100
statistics = True
```

Ignore specific messages
pylint --disable=C0114 testlint/util.py

- Make it a habit of running pylint before you commit
- with existing projects:
  - there will be a long list of things to fix
  - work at it over time; focus on single message IDs only
  - run tests when you are touching actual code


How to keep jupyter notebooks clean?

https://stackoverflow.com/questions/26126853/verifying-pep8-in-ipython-notebook-code

pip install flake8 pycodestyle_magic

In [3]:
%load_ext pycodestyle_magic

In [4]:
%pycodestyle_on

In [6]:
a=1
print(a)

1:2: E225 missing whitespace around operator


1


I noted, that this will sometimes disable cell print output.

In [10]:
a="I shall not be printed"

1:2: E225 missing whitespace around operator


To do this on the command line, one can also convert a jupyter notebook to a script and run pylint on this script for cleanup:

```
jupyter nbconvert --to=script --output-dir=/tmp/converted-notebooks/ notebook_name.ipynb
pylint --disable=C0103,C0413,C0305 /tmp/converted-notebooks/notebook_name.py
```

Here is also an interesting blogpost how jupyter notebook testing can be further automatized: https://www.blog.pythonlibrary.org/2018/10/16/testing-jupyter-notebooks/

### Matlab code checker and styleguides

Since R2011b, matlab ships its own code checker:

https://de.mathworks.com/help/matlab/ref/checkcode.html

The checker can be run against a single file:
```
checkcode(filename)
```

It will also print a list of potential errors and suggestions how to improve the code quality.

Older versions of Matlab feature a tool called "mlintrpt" that basically works identical to checkcode.

I was not able to find an official document regarding Matlab code style analogous to PEP8, but there are a couple of inofficial documents around that address certain standards that can be observed when writing Matlab code to keep it consistent and easy to read; e.g.:

http://www.cs.cornell.edu/courses/cs321/2003fa/Matlab%20Coding%20Style.pdf

<table>
<tr>
<th>Python</th>
<th>Matlab</th>
</tr>
<tr>
<td>

```python
pytest test/testfile.py
```

</td>
<td>

```matlab
import matlab.unittest.TestRunner
import matlab.unittest.TestSuite

suite = TestSuite.fromClass(?mypackage.MyTestClass);
runner = TestRunner.withTextOutput;
result = runner.run(suite)
```

</td>
</tr>
</table>
